In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split


df_train=pd.read_csv("/dbfs/FileStore/tables/train.csv")
df_test=pd.read_csv("/dbfs/FileStore/tables/test.csv")


def scale(v):
    return (v-v.min())/(v.max()-v.min())



df_X=df_train.drop("label",axis=1)

X=df_X.apply(scale)
# Dropping all the columns with only NaN values
X=X.dropna(axis=1,how='all')
# Saving the label data as the target variable
y=df_train["label"]

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import decomposition
train_x,test_x,train_y,test_y=train_test_split(X,y,test_size=0.30,random_state=42,stratify=y)



pca = decomposition.PCA(n_components=0.828)
pca.fit(train_x)
X_train_pca = pca.transform(train_x)
X_pca = pca.transform(test_x)

clf = SVC(kernel="rbf",C=35,gamma=0.01)
clf.fit(X_train_pca,train_y)
predictions=clf.predict(X_pca)
from sklearn.metrics import accuracy_score
lm= accuracy_score(test_y,predictions)
print(lm)
df_test=(df_test-df_X.min())/(df_X.max()-df_X.min())
# Only looking at the features that were present in our training set as well
X_t=df_test[X.columns]
# Doing the PCA transform
X_trans=pca.transform(X_t)


final=clf.predict(X_trans)
df=pd.DataFrame(final,columns=['Label'],index=np.arange(1,28001))
# Saving the dataframe into a result.csv file
df.to_csv('/dbfs/FileStore/tables/gridsol.csv', sep=',', header=True, index=False)
  

0.979047619047619